In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Loading Files**

In [2]:
train_file = pd.read_csv("./kaggle/input/train.csv")
test_file = pd.read_csv("./kaggle/input/test.csv")
submission_file = pd.read_csv("./kaggle/input/sample_submission.csv")

In [3]:
train_file.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
test_file.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Setting up X Y and cleaning**

In [5]:
X_train = train_file.drop(columns=['label']).values
Y_train = train_file['label'].values

X_test = test_file.values

X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

In [6]:
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [7]:
Y_train

array([1, 0, 1, ..., 7, 6, 9])

Normally you would need to flatten images and then normalize the vectors between 0 or 1 but this dataset has this applied 

In [8]:
print(X_train.shape)

(42000, 784)


**Creating Model**

In [9]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(784,)))  # Input layer with 784 input units
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))  # Output layer for 10 classes

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1050/1050 ━━━━━━━━━━━━━━━━━━━━ 1s 761us/step - accuracy: 0.8479 - loss: 0.5262 - val_accuracy: 0.9492 - val_loss: 0.1618
Epoch 2/5
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 1s 747us/step - accuracy: 0.9629 - loss: 0.1281 - val_accuracy: 0.9620 - val_loss: 0.1190
Epoch 3/5
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 1s 713us/step - accuracy: 0.9734 - loss: 0.0818 - val_accuracy: 0.9629 - val_loss: 0.1174
Epoch 4/5
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 1s 687us/step - accuracy: 0.9807 - loss: 0.0617 - val_accuracy: 0.9656 - val_loss: 0.1120
Epoch 5/5
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 1s 714us/step - accuracy: 0.9855 - loss: 0.0468 - val_accuracy: 0.9677 - val_loss: 0.1078


**Train and testing the network**

In [10]:
model.fit(X_train, Y_train, epochs=5, batch_size=32, validation_split=0.2, verbose=2)
scores = model.evaluate(X_train, Y_train, verbose=0)

Epoch 1/5
1050/1050 - 1s - 751us/step - accuracy: 0.9876 - loss: 0.0382 - val_accuracy: 0.9699 - val_loss: 0.1102
Epoch 2/5
1050/1050 - 1s - 771us/step - accuracy: 0.9890 - loss: 0.0339 - val_accuracy: 0.9705 - val_loss: 0.1000
Epoch 3/5
1050/1050 - 1s - 672us/step - accuracy: 0.9919 - loss: 0.0248 - val_accuracy: 0.9707 - val_loss: 0.1113
Epoch 4/5
1050/1050 - 1s - 750us/step - accuracy: 0.9933 - loss: 0.0205 - val_accuracy: 0.9730 - val_loss: 0.1118
Epoch 5/5
1050/1050 - 1s - 689us/step - accuracy: 0.9928 - loss: 0.0208 - val_accuracy: 0.9744 - val_loss: 0.1045


In [11]:
print('Accuracy: {}% \n Error: {}'.format(scores[1], 1 - scores[1]))        

Accuracy: 0.9928809404373169% 
 Error: 0.0071190595626831055


Saving model

In [16]:
pretrained_model = keras.saving.save_model(model, 'classification.keras')

**Submission**

In [13]:
proba=model.predict(X_test)
y_pred=np.argmax(proba,axis=1)
y_pred

875/875 ━━━━━━━━━━━━━━━━━━━━ 0s 268us/step


array([2, 0, 9, ..., 3, 9, 2])

In [14]:
submission_file["Label"]=y_pred
submission_file.to_csv("submission.csv",index=False)

In [15]:
submission_file

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9
